In [1]:
# !pip3 install bert-tensorflow==1.0.1
# !wget https://f000.backblazeb2.com/file/malay-dataset/corpus/goemotions/goemotions_1.translated.csv
# !wget https://f000.backblazeb2.com/file/malay-dataset/corpus/goemotions/goemotions_2.translated.csv
# !wget https://f000.backblazeb2.com/file/malay-dataset/corpus/goemotions/goemotions_3.translated.csv
# !wget https://f000.backblazeb2.com/file/malaya-model/pretrained/fastformer-base-social-media-2021-11-02.tar.gz
# !tar -zxf fastformer-base-social-media-2021-11-02.tar.gz
# !rm fastformer-base-social-media-2021-11-02.tar.gz

In [2]:
import pandas as pd
from glob import glob
from tqdm import tqdm

texts, labels = [], []

files = glob('goemotions_*.translated.csv')
for file in files:
    print(file)
    df = pd.read_csv(file)
    for i in tqdm(range(len(df))):
        row = df.iloc[i]
        label = row.iloc[9:-2].tolist()
        t = row.iloc[[0, -2, -1]].tolist()
        texts.extend(t)
        labels.extend([label] * len(t))

goemotions_3.translated.csv


100%|██████████| 71225/71225 [00:07<00:00, 9574.98it/s]


goemotions_1.translated.csv


100%|██████████| 70000/70000 [00:07<00:00, 9644.32it/s]


goemotions_2.translated.csv


100%|██████████| 70000/70000 [00:07<00:00, 9582.44it/s]


In [3]:
len(texts), len(labels)

(633675, 633675)

In [4]:
import random
from unidecode import unidecode
import re

normalized_chars = {}

chars = '‒–―‐—━—-▬'
for char in chars:
    normalized_chars[ord(char)] = '-'

chars = '«»“”¨"'
for char in chars:
    normalized_chars[ord(char)] = '"'

chars = "’'ʻˈ´`′‘’\x92"
for char in chars:
    normalized_chars[ord(char)] = "'"

chars = '̲_'
for char in chars:
    normalized_chars[ord(char)] = '_'

chars = '\xad\x7f'
for char in chars:
    normalized_chars[ord(char)] = ''

chars = '\n\r\t\u200b\x96'
for char in chars:
    normalized_chars[ord(char)] = ' '

    
laughing = {
    'huhu',
    'haha',
    'gagaga',
    'hihi',
    'wkawka',
    'wkwk',
    'kiki',
    'keke',
    'huehue',
    'hshs',
    'hoho',
    'hewhew',
    'uwu',
    'sksk',
    'ksks',
    'gituu',
    'gitu',
    'mmeeooww',
    'meow',
    'alhamdulillah',
    'muah',
    'mmuahh',
    'hehe',
    'salamramadhan',
    'happywomensday',
    'jahagaha',
    'ahakss',
    'ahksk'
}

def make_cleaning(s, c_dict):
    s = s.translate(c_dict)
    return s

def cleaning(string):
    """
    use by any transformer model before tokenization
    """
    string = unidecode(string)
    
    string = ' '.join(
        [make_cleaning(w, normalized_chars) for w in string.split()]
    )
    string = re.sub('\(dot\)', '.', string)
    string = (
        re.sub(re.findall(r'\<a(.*?)\>', string)[0], '', string)
        if (len(re.findall(r'\<a (.*?)\>', string)) > 0)
        and ('href' in re.findall(r'\<a (.*?)\>', string)[0])
        else string
    )
    string = re.sub(
        r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', ' ', string
    )
    
    chars = '.,/'
    for c in chars:
        string = string.replace(c, f' {c} ')
        
    string = re.sub(r'[ ]+', ' ', string).strip().split()
    string = [w for w in string if w[0] != '@']
    x = []
    for word in string:
        word = word.lower()
        if any([laugh in word for laugh in laughing]):
            if random.random() >= 0.5:
                x.append(word)
        else:
            x.append(word)
    string = [w.title() if w[0].isupper() else w for w in x]
    return ' '.join(string)

In [5]:
X, Y = [], []
for i in tqdm(range(len(texts))):
    try:
        c = cleaning(texts[i])
        if len(c) > 2:
            X.append(c)
            Y.append(labels[i])
    except:
        pass
    
len(X), len(Y)

100%|██████████| 633675/633675 [00:16<00:00, 38303.55it/s]


(633633, 633633)

In [6]:
from malaya.text.bpe import WordPieceTokenizer

tokenizer = WordPieceTokenizer('BERT.wordpiece', do_lower_case = False)
# tokenizer.tokenize('halo nama sayacomel')

2021-11-17 12:56:56.535319: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/resource_loader.py:78: UserWarning: You are currently using TensorFlow 2.5.0 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops against TensorFlow 2.6.0, and there are no compatibility guarantees between the two versions. 
This means that you might get segfaults when loading the custom op, or other kind of low-level errors.
 If you do, do not file an issue on Github. This is a known limitation.

It might help you to fallback to pure Python ops by setting environment variable `TF_ADDONS_PY_OPS=1` or using `tfa.options.disable_custom_kernel()` in your code. To do that, see https://github.com/tensorflow/addons#gpucpu-custom-ops 

You can also change the TensorFlow version installed on your system. You would 

In [7]:
input_ids, input_masks = [], []
actual_l = []

maxlen = 200
for i in tqdm(range(len(X))):
    text = X[i]
    tokens_a = tokenizer.tokenize(text)
    tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
    input_id = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_id)
    
    if len(input_id) <= maxlen:
        input_ids.append(input_id)
        input_masks.append(input_mask)
        actual_l.append(Y[i])

100%|██████████| 633633/633633 [00:52<00:00, 12116.98it/s]


In [8]:
len(input_ids), len(actual_l)

(633612, 633612)

In [9]:
import pickle

with open('goemotions-fastformer.pkl', 'wb') as fopen:
    pickle.dump([input_ids, actual_l], fopen)